# Deep Learning

In this lab you train deep learning models built with Keras and a Tensorflow backend on Azure Databricks cluster whose nodes provide GPUs.

## Train an autoencoder using GPU

In the following cell you will use Keras to define an autoencoder. 

*Don't get hung up on the details of constructing the auto-encoder. The point of this lab is to show you how to train neural networks using GPU's.* 

Execute the following cell. In the output, verify that `K.tensorflow_backend._get_available_gpus()` returned an entry describing a GPU available in your environment.

In [5]:
# Step 1 - Train an autoencoder using GPU
#########################################
import numpy as np
import keras
from keras import backend as K
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.utils.vis_utils import plot_model
import azureml
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.model import Model as AMLModel
import pickle

# Verify we have a GPU available
# The output of the following should not be an empty array
# If you get an empty array back, it means no GPU was detected, which might mean you need to 
# uninstall keras/tensorflow/tensorflow-gpu and then reinstall tensorflow-gpu and keras
K.tensorflow_backend._get_available_gpus()

# We use Fashion mnist dataset
from keras.datasets import fashion_mnist

# We download and load the data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


# Build the encoder
input_img = Input(shape=(28, 28, 1)) # This will be the input 👚

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded_feature_vector = MaxPooling2D((2, 2), padding='same', name='feature_vector')(x) #  <---- name your feature vector somehow

# at this point the representation is (4, 4, 8) i.e. 128-dimensional compressed feature vector

# Build the decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded_feature_vector)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded_output = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)


# The first model is autoencoder model, it takes the input image and results in a decoded image
autoencoder_model = Model(input_img, decoded_output)
# Compile the first model
autoencoder_model.compile(optimizer='adadelta', loss='binary_crossentropy')


# The second NN model is only a half of the first model, it take the input image and gives the encoded vector as output
encoder_model = Model(inputs=autoencoder_model.input,
                                 outputs=autoencoder_model.get_layer('feature_vector').output) # <---- take the output from the feature vector
# Compile the second model
encoder_model.compile(optimizer='adadelta', loss='binary_crossentropy')

# We need to scale the image from [0-255] to [0-1] for better performance of activation functions
x_train = x_train / 255.
x_test = x_test / 255.


# We train the NN in batches (groups of images), so we reshape the dataset
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

print("Train dataset size is {0}".format(x_train.shape))
print("Test dataset size is {0}".format(x_test.shape))

Using TensorFlow backend.
Downloading data from http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz

 8192/29515 [=======>......................] - ETA: 0s
16384/29515 [===============>..............] - ETA: 0s
32768/29515 [=================================] - 0s 5us/step
Downloading data from http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz

 8192/26421880 [..............................] - ETA: 0s
 16384/26421880 [..............................] - ETA: 3:58
 49152/26421880 [..............................] - ETA: 2:39
 106496/26421880 [..............................] - ETA: 1:49
 229376/26421880 [..............................] - ETA: 1:07
 466944/26421880 [..............................] - ETA: 41s 
 950272/26421880 [>.............................] - ETA: 23s
 1908736/26421880 [=>............................] - ETA: 12s
 1916928/26421880 [=>............................] - ETA: 13s
 3833856/26421880 [===>..........................] - ETA: 6s 
 6856704/26421880 [======>.......................] - ETA: 3s
 6979584/26421880 [======>.......................] - ETA: 3s
 9977856/26421880 [==========>...................] - ETA: 2s
10117120/26421880 [==========>...................] - ETA: 2s
13107200/26421880 [=============>................] - ETA: 1s
13238272/26421880 [==============>...............] - ETA: 1s
15900672/26421880 [=================>............] - ETA: 1s
16285696/26421880 [=================>............] - ETA: 1s
18620416/26421880 [====================>.........] - ETA: 0s
19144704/26421880 [====================>.........] - ETA: 0s
21741568/26421880 [=======================>......] - ETA: 0s
21962752/26421880 [=======================>......] - ETA: 0s
24666112/26421880 [===========================>..] - ETA: 0s
25067520/26421880 [===========================>..] - ETA: 0s
26427392/26421880 [==============================] - 2s 0us/step
Downloading data from http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz

8192/5148 [===============================================] - 0s 0us/step
Downloading data from http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz

 8192/4422102 [..............................] - ETA: 0s
 16384/4422102 [..............................] - ETA: 39s
 49152/4422102 [..............................] - ETA: 26s
 106496/4422102 [..............................] - ETA: 17s
 229376/4422102 [>.............................] - ETA: 10s
 466944/4422102 [==>...........................] - ETA: 6s 
 950272/4422102 [=====>........................] - ETA: 3s
1916928/4422102 [============>.................] - ETA: 1s
3833856/4422102 [=========================>....] - ETA: 0s
4423680/4422102 [==============================] - 1s 0us/step
Train dataset size is (60000, 28, 28, 1)
Test dataset size is (10000, 28, 28, 1)

Once you have your autoencoder model structured, you need to train the the underlying neural network. Training this model on regular CPU's will take 10s of minutes to hours. However, you can execute this same code in an environment with GPU's for better performance. Execute the following cell. How long did your training take?

In [7]:
# Step 2 - Train a neural network
#################################
# It takes several minutes to train this neural network, depending on the configuration of your cluster.
learning_history=autoencoder_model.fit(x=x_train, y=x_train, epochs=10, batch_size=128, 
                                 shuffle=True, validation_data=(x_test, x_test), verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10

 128/60000 [..............................] - ETA: 5:24 - loss: 0.6911
 256/60000 [..............................] - ETA: 3:13 - loss: 0.6892
 384/60000 [..............................] - ETA: 2:29 - loss: 0.6871
 512/60000 [..............................] - ETA: 2:07 - loss: 0.6842
 640/60000 [..............................] - ETA: 1:54 - loss: 0.6805
 768/60000 [..............................] - ETA: 1:45 - loss: 0.6764
 896/60000 [..............................] - ETA: 1:39 - loss: 0.6729
 1024/60000 [..............................] - ETA: 1:34 - loss: 0.6673
 1152/60000 [..............................] - ETA: 1:30 - loss: 0.6642
 1280/60000 [..............................] - ETA: 1:27 - loss: 0.6603
 1408/60000 [..............................] - ETA: 1:24 - loss: 0.6563
 1536/60000 [..............................] - ETA: 1:22 - loss: 0.6518
 1664/60000 [..............................] - ETA: 1:20 - loss: 0.6472
 1792/60000 [..............................] - ETA: 1:19 - loss: 0.6416
 1920/60000 [..............................] - ETA: 1:17 - loss: 0.6360
 2048/60000 [>.............................] - ETA: 1:16 - loss: 0.6315
 2176/60000 [>.............................] - ETA: 1:15 - loss: 0.6259
 2304/60000 [>.............................] - ETA: 1:14 - loss: 0.6210
 2432/60000 [>.............................] - ETA: 1:13 - loss: 0.6152
 2560/60000 [>.............................] - ETA: 1:12 - loss: 0.6121
 2688/60000 [>.............................] - ETA: 1:11 - loss: 0.6057
 2816/60000 [>.............................] - ETA: 1:10 - loss: 0.5999
 2944/60000 [>.............................] - ETA: 1:09 - loss: 0.5944
 3072/60000 [>.............................] - ETA: 1:09 - loss: 0.5887
 3200/60000 [>.............................] - ETA: 1:08 - loss: 0.5839
 3328/60000 [>.............................] - ETA: 1:08 - loss: 0.5812
 3456/60000 [>.............................] - ETA: 1:07 - loss: 0.5806
 3584/60000 [>.............................] - ETA: 1:06 - loss: 0.5760
 3712/60000 [>.............................] - ETA: 1:06 - loss: 0.5717
 3840/60000 [>.............................] - ETA: 1:06 - loss: 0.5675
 3968/60000 [>.............................] - ETA: 1:05 - loss: 0.5634
 4096/60000 [=>............................] - ETA: 1:05 - loss: 0.5603
 4224/60000 [=>............................] - ETA: 1:05 - loss: 0.5567
 4352/60000 [=>............................] - ETA: 1:04 - loss: 0.5535
 4480/60000 [=>............................] - ETA: 1:04 - loss: 0.5498
 4608/60000 [=>............................] - ETA: 1:04 - loss: 0.5470
 4736/60000 [=>............................] - ETA: 1:03 - loss: 0.5435
 4864/60000 [=>............................] - ETA: 1:03 - loss: 0.5398
 4992/60000 [=>............................] - ETA: 1:03 - loss: 0.5365
 5120/60000 [=>............................] - ETA: 1:02 - loss: 0.5339
 5248/60000 [=>............................] - ETA: 1:02 - loss: 0.5311
 5376/60000 [=>............................] - ETA: 1:03 - loss: 0.5288
 5504/60000 [=>............................] - ETA: 1:03 - loss: 0.5261
 5632/60000 [=>............................] - ETA: 1:02 - loss: 0.5235
 5760/60000 [=>............................] - ETA: 1:02 - loss: 0.5213
 5888/60000 [=>............................] - ETA: 1:02 - loss: 0.5188
 6016/60000 [==>...........................] - ETA: 1:01 - loss: 0.5168
 6144/60000 [==>...........................] - ETA: 1:01 - loss: 0.5147
 6272/60000 [==>...........................] - ETA: 1:01 - loss: 0.5127
 6400/60000 [==>...........................] - ETA: 1:01 - loss: 0.5108
 6528/60000 [==>...........................] - ETA: 1:00 - loss: 0.5085
 6656/60000 [==>...........................] - ETA: 1:00 - loss: 0.5061
 6784/60000 [==>...........................] - ETA: 1:00 - loss: 0.5042
 6912/60000 [==>...........................] - ETA: 1:00 - loss: 0.5026
 7040/60000 [==>...........................] - ETA: 59s -

With a trained auto-encoder in hand, try using the model by executing the following cell.

In [9]:
# Step 3 - Test the model
##########################
encoded_decoded_image=autoencoder_model.predict(x_test)

## Register the neural network model with Azure Machine Learning

In the following cell, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments to create or retrieve your workspace. Observe that you can register a neural network model with Azure Machine Learning in exactly the same way you would register a classical machine learning model. Execute the cell to register the model.

In [12]:
# Step 4 - Export and Register the model
########################################

# Create a new Workspace or retrieve the existing one
#Provide the Subscription ID of your existing Azure subscription
subscription_id ='757c4165-0823-49f7-9678-5a85fe5e13cc'

# Provide values for the Resource Group and Workspace that will be created
resource_group = 'MLworkspace2'
workspace_name = 'snml2'
workspace_region = 'westeurope'  # eastus, westcentralus, southeastasia, australiaeast, westeurope
# By using the exist_ok param, if the worskpace already exists we get a reference to the existing workspace instead of an error
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

print("Workspace Provisioning complete.")

# Serialize the model to a pickle file in the outputs folder
model_name = 'autoencoder'
import os
if os.path.exists('outputs') is False:
  os.mkdir('outputs')
output_model_path = 'outputs/' + model_name + '.pkl'
pickle.dump(learning_history,open(output_model_path,'wb'))
print('Exported model to ', output_model_path)

# notice for the model_path, we supply the name of the outputs folder without a trailing slash
registered_model = AMLModel.register(model_path='outputs', model_name=model_name, workspace=ws)

Workspace Provisioning complete.
Exported model to outputs/autoencoder.pkl
Registering model autoencoder